In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.monitoring.prefect_logs" #features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path)#, limit=200000)

In [4]:
max_date = df["timestamp"].max()
max_date

In [24]:
df.to_csv(f"DataChecks/prefect_logs/prefect_logs_{max_date}.csv", index=False)

In [6]:
temp = df[df["node_name"].isin(["x4llarge"])].sort_values(by="timestamp")
temp = df[df["timestamp"] >= "2025-09-01"].sort_values(by="timestamp")
temp = temp[temp["timestamp"] <= "2025-09-04"].sort_values(by="timestamp")


temp

In [7]:
df["node_name"].value_counts()

In [8]:
df["name"].value_counts()

In [9]:
df["date"] = df["timestamp"].dt.date

In [10]:
range1 = df[(df["timestamp"]>="2025-07-16")&(df["timestamp"]<="2025-08-25")]
range2 = df[(df["timestamp"]>="2025-08-04")&(df["timestamp"]<="2025-09-22")]
print(range1["timestamp"].dt.date.value_counts().sort_index())
print(range2["timestamp"].dt.date.value_counts().sort_index())

In [11]:
range1["name"].value_counts().sort_index()

In [12]:
range1.groupby("date")["name"].value_counts().sort_index()

In [13]:
range2["name"].value_counts().sort_index()

In [14]:
range2.groupby("date")["name"].value_counts().sort_index()

# Get Node sizes

In [15]:
node_names1 = range1.groupby(by=["name","account","date"])["node_name"].value_counts().sort_index().reset_index()
node_names1

In [16]:
node_names2 = range2.groupby(by=["name","account","date"])["node_name"].value_counts().sort_index().reset_index()
node_names2

In [17]:
node_names = pd.merge(node_names1,node_names2, how="outer", on=["name","account","date","node_name"], suffixes=["_1","_2"])
node_names = node_names.sort_values(by=["name","account","date"])
node_names

# Node SIzes for Targeting

In [18]:
targeting1 = range1[range1["name"]=="k8-targeting"].groupby(by=["account","audience_id","targeting_outlook","date"])["node_name"].nunique().reset_index().rename(columns={"node_name":"count"})
targeting1_names = range1[range1["name"]=="k8-targeting"].groupby(by=["account","audience_id","targeting_outlook","date"])["node_name"].unique().reset_index()
targeting1 = pd.merge(targeting1,targeting1_names, how="outer", on=["account","audience_id","targeting_outlook","date"]).sort_values(by=["account","audience_id","date"]).reset_index()
targeting1

In [19]:
targeting2 = range2[range2["name"]=="k8-targeting"].groupby(by=["account","audience_id","targeting_outlook","date"])["node_name"].nunique().reset_index().rename(columns={"node_name":"count"})
targeting2_names = range2[range2["name"]=="k8-targeting"].groupby(by=["account","audience_id","targeting_outlook","date"])["node_name"].unique().reset_index()
targeting2 = pd.merge(targeting2,targeting2_names, how="outer", on=["account","audience_id","targeting_outlook","date"]).sort_values(by=["account","audience_id","date"]).reset_index()
targeting2

In [20]:
targeting = pd.merge(targeting1,targeting2, how="outer", on=["account","audience_id","targeting_outlook","date"],suffixes=("_1","_2"))
targeting[targeting["audience_id"].isin(targeting[targeting["date"].astype("string")=="2025-08-07"]["audience_id"].unique().tolist())]

In [21]:
targeting["same"] = targeting[["node_name_1","node_name_2"]].apply(lambda x: (x[0] == x[1]).all(), axis=1)
targeting


In [22]:
targeting

In [23]:
from datetime import datetime
temp = targeting[targeting["same"] == False].dropna(subset=["count_1"])
temp = temp.dropna(subset=["count_2"])
temp.to_csv(f"DataChecks/prefect_logs/check_prefect_logs_{datetime.today()}.csv")
temp